## Hierarchical Indexing

In [1]:
import pandas as pd
import numpy as np

#### The bad way

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

#### Using Pandas MultiIndex

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [7]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

#### MultiIndex as extra dimension

In [8]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [9]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [10]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [12]:
f_u18 = 100*pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,27.359428,24.921096
New York,24.700997,22.283054
Texas,28.325110,27.356773


#### Methods of MultiIndex Creation

In [13]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.725346  0.966829
  2  0.566462  0.888198
b 1  0.535910  0.111810
  2  0.492355  0.723353

In [14]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### MultiIndex for columns

In [15]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      48.0  34.5  51.0  38.0  19.0  37.2
     2      38.0  38.4  30.0  38.0  35.0  39.1
2014 1      24.0  36.8  54.0  35.9  28.0  35.7
     2      31.0  36.4  58.0  37.0  40.0  36.4

In [16]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      51.0  38.0
     2      30.0  38.0
2014 1      54.0  35.9
     2      58.0  37.0

### Indexing and Slicing a MultiIndex

In [17]:
pop['California', 2000]

33871648

In [18]:
pop[pop > 22000000]

California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [19]:
health_data['Guido', 'HR']

year  visit
2013  1        51.0
      2        30.0
2014  1        54.0
      2        58.0
Name: (Guido, HR), dtype: float64

In [20]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      48.0  34.5
     2      38.0  38.4

In [21]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        48.0
      2        38.0
2014  1        24.0
      2        31.0
Name: (Bob, HR), dtype: float64

In [24]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,48.0,51.0,19.0
2014,1,24.0,54.0,28.0


### Stacking and unstacking indices

In [25]:
pop.unstack(level=0)

,California,New York,Texas
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [27]:
pop.unstack(level=1)

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [29]:
# The opposite of unstack() is stack(), which here can be used to recover the original series:
pop.unstack().stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Index setting and resetting

In [32]:
pop_flat = pop.reset_index(name='population')
pop_flat

,level_0,level_1,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


### Data Aggregations on Multi-Indices

In [34]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      48.0  34.5  51.0  38.0  19.0  37.2
     2      38.0  38.4  30.0  38.0  35.0  39.1
2014 1      24.0  36.8  54.0  35.9  28.0  35.7
     2      31.0  36.4  58.0  37.0  40.0  36.4

In [35]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     43.0  36.45  40.5  38.00  27.0  38.15
2014     27.5  36.60  56.0  36.45  34.0  36.05

In [36]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,36.833333,37.533333
2014,39.166667,36.366667
